In [6]:
import pandas as pd
import datetime
import requests
import json
import sys, os
import itertools
import time
import numpy as np
from os import path
from pathlib import Path
from binance.spot import Spot
from twelvedata import TDClient
from sklearn import metrics
from sklearn.inspection import permutation_importance
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import itertools
import xgboost as xgb
import seaborn as sns
from typing import Union, Dict, List
import talib
import ta
import shap
from coinmetrics.api_client import CoinMetricsClient
sys.path.insert(0, os.path.join(os.getcwd(), ".."))
import fin_utilities 
import sklearn
import scikitplot as skplt
import matplotlib.pyplot as plt
cfg = fin_utilities.__cfg_reading("pred")
import my_functions
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, Dense,BatchNormalization
from keras.layers import Dropout, Activation
from sklearn.preprocessing import MinMaxScaler
from keras.layers import TimeDistributed


import matplotlib as mpl
# Set the default color cycle
mpl.rcParams.update(mpl.rcParamsDefault)


#td = TDClient(apikey=cfg['TWELVEDATA']['API'])  

PROJECT_DIR = eval(cfg['PROJECT_PATH'])
DATA_PATH   = PROJECT_DIR / cfg['DATA_FOLDER']
SOURCE      = cfg['API_DATA_SOURCE']
URL         = cfg[SOURCE]['API_URL_HIST_DATA']
COLUMNS     = cfg[SOURCE]['COLUMN_NAMES']
PARAMS      = cfg[SOURCE]['REQ_PARAMS']
SYMBOL      = cfg['SYMBOL']
STABLECOIN  = cfg['STABLECOIN']
MAX_LENGTH  = cfg['MAX_TRADE_DURATION']
XGB_PARAM   = cfg['xgb']
TEST_SIZE   = 100
client = CoinMetricsClient()
print(client)


In [ ]:
def extract_values(dict_list, key):
    """
    Extracts the values associated with a given key from a list of dictionaries.

    Args:
        dict_list (list): A list of dictionaries.
        key (str): The key to extract the value from.

    Returns:
        list: A list of values associated with the key.
    """
    values = []
    for dictionary in dict_list:
        if key in dictionary:
            values.append(dictionary[key])
    return values


def create_dict(list1, list2):
    """
    Given two lists, create a dictionary with keys from list1 and values from list2

    Args:
        list1 (list): The list of keys for the dictionary.
        list2 (list): The list of values for the dictionary.

    Returns:
        dict: A dictionary with keys from list1 and values from list2, intercepted with available metrics
    """
    result_dict = {}
    for key in list1:
        result_dict[key] = list(set(extract_values(client.catalog_assets(key)[0]['metrics'], 'metric')) & set(list2))
    return result_dict


In [ ]:
SYMBOL = ['btc']

frequency  = "1d"
asset      = SYMBOL
metrics_a = [
    #'CapAct1yrUSD',
    'CapMVRVCur',
    'CapMVRVFF',
    #'CapMrktCurUSD',
    #'CapMrktEstUSD',
    #'CapMrktFFUSD',
    'CapRealUSD',
    'AdrActCnt',
    'FeeTotUSD',
    'FeeMeanUSD',
    'DiffLast',
    'DiffMean',
    'NDF',
    'TxCnt',
    'TxCntSec',
    #'TxTfrCnt'
]


d = create_dict(SYMBOL, metrics_a)

In [ ]:
df_list = []

for asset, metrics_asset in d.items():
    df_list.append(client.get_asset_metrics(
                                    assets    = asset,
                                    metrics   = metrics_asset,
                                    frequency = frequency,
                                    start_time= PARAMS['startTime'],
                                    end_time  = PARAMS['endTime'],
                                    page_size = 10000
                                ).to_dataframe()
    )

df_mcap = pd.concat(df_list, axis=0, ignore_index=True)


#droppo tutte le colonne "status" inutili
df_mcap.drop(columns=[col for col in df_mcap.columns if 'status' in col], inplace=True)

#replace di tutti i campi None stringa a -1
for col in df_mcap.columns[2:]:
    if len(df_mcap[df_mcap[col]=='None'])!=0:
        df_mcap[col]=df_mcap[col].replace('None','-1')
    else:
        df_mcap[col]=df_mcap[col].fillna(-1)

df_mcap = df_mcap.astype('float64', errors='ignore')
df_mcap = my_functions.round_float_cols(df_mcap)

#df_mcap = df_mcap[df_mcap['AdrActCnt']!=0].reset_index(drop=True)
#df_mcap = df_mcap[df_mcap['AdrActCnt']!=None].reset_index(drop=True)

In [ ]:
my_functions.plt_correlation(df_mcap)

In [ ]:
# EXPORT DI TUTTI I DATI DA TUTTI GLI EXCHANGE
from typing import Optional
import logging

logger = logging.getLogger()
stream_handler = logging.StreamHandler()
level = logging.getLevelName("INFO")
stream_handler.level = level
formatter = logging.Formatter(
    datefmt="[%Y-%m-%d %H:%M:%S]", fmt="%(asctime)-15s %(levelname)s %(message)s"
)
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.level = level

# use it if you want to get specific exchanges or leave it empty if you want to get all exchanges data
EXCHANGES_TO_EXPORT = {}

# use it if you want to get specific markets or leave it empty if you want to get all markets
# example of market name to be used in this filter: "binance-BTCUSDT-future",
# note that if you specified exchanges filter, it will act as selecting intersection with the markets to export
# not as union.
MARKETS_TO_EXPORT = {}

# example values: "spot", "future", "option"
# you can use all 3 if you want or just a subset
MARKET_TYPES_TO_COLLECT = {
    "spot"   
}

# leave it empty to catch all
BASE_MARKETS = {
    "btc",
}

# leave it empty to catch all
QUOTE_MARKETS = {
    "usdt",
}

# 1m, 5m, 10m, 15m, 30m, 1h, 4h, 1d
FREQUENCY = "1d"

DST_ROOT = "./data"
EXPORT_START_DATE = "2009-01-01"
EXPORT_END_DATE: Optional[str] = None
# path to local file that is used to not reexport data if it was already exported
PROCESSED_DAYS_REGISTRY_FILE_PATH = "candles_processed_days_registry.txt"

client = CoinMetricsClient()

if __name__ == "__main__":
    export_start_time = datetime.datetime.now()
    try:
        my_functions.export_data(
                client,
                PROCESSED_DAYS_REGISTRY_FILE_PATH, 
                EXCHANGES_TO_EXPORT,
                MARKETS_TO_EXPORT,
                MARKET_TYPES_TO_COLLECT,
                BASE_MARKETS,
                QUOTE_MARKETS,
                EXPORT_START_DATE,
                str(datetime.datetime.now().date()), 
                DST_ROOT, 
                FREQUENCY 
        )
    finally:
        print("export took: %s", datetime.datetime.now() - export_start_time)


mkts = my_functions.get_markets_to_process(client,EXCHANGES_TO_EXPORT, MARKETS_TO_EXPORT, MARKET_TYPES_TO_COLLECT, BASE_MARKETS, QUOTE_MARKETS)

df = pd.DataFrame()

for mkt in mkts:
    print(mkt) 
    
    market_data_root = "/".join(                                                                                                                                    
                (                                                                                                                                                        
                    DST_ROOT.rstrip("/"),                                                                                                                                
                    mkt["market"].split("-")[0],                                                                                                                         
                    my_functions.get_instrument_root(mkt),                                                                                                                            
                ) 
    )      
    print(f'manng {my_functions.get_instrument_root(mkt)}')

    df = pd.concat([df,
                    my_functions.read_data_for_a_market(
                                market_data_root,                                                                                                                                                               
                                FREQUENCY                                         
                            )
                    ],
                     ignore_index=True)       

#max volums for each day between all the exchanges
df = df[df['volume'] == df.groupby('time')['volume'].transform('max')].sort_values(by='time').reset_index(drop=True)
df['time'] = pd.to_datetime(df['time']).dt.date
df['time'] = pd.to_datetime(df['time'])
df['market'] = df['market'].str.split('-',2, expand=True)[1]
df.head(5)

In [ ]:
dxy = pd.read_csv('data/DXY.csv')
dxy = dxy[['Date','Adj Close']]
dxy['Date'] = pd.to_datetime(dxy['Date'])
#inserisco date mancanti
dxy = pd.merge(pd.Series(pd.date_range(dxy['Date'].min(),dxy['Date'].max(),freq='d'), name='Date'), dxy, on='Date', how='left').reset_index(drop=True)
dxy['Dxy_Close'] = np.round(dxy['Adj Close'].ffill(),2)
dxy.drop(columns='Adj Close', inplace=True)
df = pd.merge(df,dxy,left_on='time',right_on='Date',how='left')
df.drop(columns=['Date'],inplace=True)
df

In [ ]:
map_ohlc = {'Open':'price_open','High':'price_high','Low':'price_low','Close':'price_close', 'Volume': 'volume', 'Number of trades': 'candle_trades_count'}
lags_relevant = [13,30,99,200]

In [ ]:
print(df.columns)
df['AVG_Candle_Price'] = (df[map_ohlc['Open']]+df[map_ohlc['High']]+df[map_ohlc['Low']]+df[map_ohlc['Close']])/4
for lag in lags_relevant:
    df['Close_lag_'+str(lag)+'d'] = df[map_ohlc['Close']].shift(lag)
    df['SMA_Close_'+str(lag)+'d'] = my_functions.rolling_kpi(df,map_ohlc['Close'],lag,'mean', False)
    df[map_ohlc['Number of trades']+'_'+str(lag)+'d'] = my_functions.rolling_kpi(df,map_ohlc['Number of trades'], lag, 'mean', False)
    df['STD_Close_'+str(lag)+'d'] = my_functions.rolling_kpi(df,map_ohlc['Close'],lag,'std', False)
    df['VAR_Close_lag_'+str(lag)+'d'] = df['STD_Close_'+str(lag)+'d']**2

In [ ]:
df = my_functions.labelize_output_according_criterion2(df, map_ohlc, max_trade_length=MAX_LENGTH)
df['signal'] = (df['min_above'].lt(df['min_below'])) & (df['min_above'] <= MAX_LENGTH)
df['signal']=df['signal'].astype(int)
df.drop(columns=['CloseAbovethreshold', 'HighAbovethreshold',
       'LowAbovethreshold', 'OpenAbovethreshold', 'CloseBelowthreshold',
       'HighBelowthreshold', 'LowBelowthreshold', 'OpenBelowthreshold', 'TP',
       'SL', 'min_above', 'min_below'], inplace=True, errors='ignore')
df = df.dropna().reset_index(drop=True)
df.head()

In [ ]:
from sklearn.preprocessing import PowerTransformer

def yeojohntrans(df, feature):   # function to apply transformer and check the distribution with histogram and kdeplot
    
    yeojohnTr = PowerTransformer(standardize=True)   # not using method attribute as yeo-johnson is the default

    plt.figure(figsize=(15,6))
    plt.subplot(1,2,1)
    plt.title("Distribution before Transformation", fontsize=15)
    sns.histplot(df[feature], kde=True, color="red")
    plt.subplot(1,2,2)
    
    df_yeojohn = pd.DataFrame(yeojohnTr.fit_transform(df[feature].values.reshape(-1,1)))
    plt.title("Distribution after Transformation", fontsize=15)
    sns.histplot(df_yeojohn,bins=20, kde=True , legend=False)
    plt.xlabel(feature)
    plt.show()
    
    print(f"Skewness was {round(df[feature],2)} before & is {round(df_yeojohn.skew()[0],2)} after Yeo-johnson transformation.")
    

In [ ]:
df_analysis = df.copy()

features = [col for col in df_analysis.columns if col not in ['market','time','signal']]
fig, axs = plt.subplots(nrows=int(len(features)/3), ncols=3, figsize=(18,5*int(len(features)/3)))
for i,feature in enumerate(features):
    df_analysis[feature] = df_analysis[feature] - df_analysis[feature].shift()
    single_feature = df_analysis[feature][1:]
    sns.histplot(data=single_feature,kde=True, stat='density', ax=axs[int(i/3)][i%3])

plt.show()


In [ ]:
df_btc = df.copy()
df_btc.dropna(inplace=True)
df_btc.drop(columns='market', inplace=True)
df_btc = df_btc.reset_index(drop=True)
df_btc.head()

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
#df_btc.drop(columns=['price_close','price_high','price_low','candle_usd_volume', 'vwap','volume','candle_trades_count','Dxy_Close'], inplace=True)
df_btc

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
features = [col for col in df_btc.columns if col not in ['time','signal']]
for feature in features:
     df_btc[feature] = df_btc[feature] - df_btc[feature].shift()
     df_btc[feature] = np.sqrt(np.absolute(df_btc[feature] - df_btc[feature].shift().fillna(0)))*np.sign(df_btc[feature] - df_btc[feature].shift().fillna(0))

df_btc = df_btc.iloc[1:]

df_btc = my_functions.round_float_cols(df_btc)
df_btc = df_btc.sample(frac=1).reset_index(drop=True)

scaler.fit(df_btc[features][:-TEST_SIZE])

X_train = df_btc[features][:-TEST_SIZE]
X_test = df_btc[features][-TEST_SIZE:]
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
Y_train = np.array(df_btc['signal'][:-TEST_SIZE])
Y_test = np.array(df_btc['signal'][-TEST_SIZE:])

#X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
#X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
p = {  
    'eval_metric':'auc',
    'tree_method':'hist',
    'lambda':.01,
    'max_depth':7,
    'scale_pos_weight':5,
    'objective':'binary:logistic',
    'colsample_bytree':.9,
    'eta':0.1,
    'n_estimators':3000,
    'early_stopping_rounds':200
    }

model = my_functions.model_selection('xgb',p)

print(model.get_params())

model.fit(X_train[:-100], Y_train[:-100], eval_set = [(X_train[:-100], Y_train[:-100],), (X_train[-100:], Y_train[-100:],)], verbose=0)
predictions = model.predict_proba(X_train[-100:])[:,1]
fpr_test, tpr_test, _ = sklearn.metrics.roc_curve(Y_train[-100:], predictions)
print("Test set score: {:.5f}".format(sklearn.metrics.auc(fpr_test, tpr_test)))



BUILD LSTM

In [ ]:
from keras.callbacks import EarlyStopping

from keras.layers.core import Dense, Dropout
from keras.optimizers import Adagrad


opt = tf.keras.optimizers.experimental.SGD(learning_rate=.2, weight_decay=0)


model = Sequential()
model.add(LSTM(50, return_sequences=True, activation='relu', input_shape=(X_train.shape[1],1), kernel_initializer=initializers.RandomNormal(stddev=0.1)))
model.add(LSTM(50, return_sequences=False, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(
            optimizer=opt,
            loss=['binary_crossentropy'], 
            metrics=['AUC','accuracy']
)


early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)

history = model.fit(X_train, Y_train, batch_size=50, epochs=5000, validation_split=0.1, shuffle=True)


# list all data in history
print(history.history.keys())
# summarize history for accuracy    
plt.plot(history.history['auc'])
plt.title('model AUC')
plt.ylabel('AUC')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
y_pr = model.predict(X_train)
y_pr

In [ ]:
model2 = Sequential()
model2.add(LSTM(units=1024, kernel_initializer='random_normal', return_sequences=True, input_shape=(X_train.shape[1], 1)))
model2.add(Dropout(0.4))
model2.add(LSTM(64))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.4))
model2.add(Dense(1, activation='sigmoid'))

# Compile the model
model2.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model2.summary()

# Fit the model to the data
history = model2.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
model3 = Sequential()
model3.add(LSTM(1025, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model3.add(LSTM(256, return_sequences=True))
model3.add(LSTM(128, return_sequences=False))   
model3.add(Dense(1, activation='sigmoid'))

model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(1, X_train.shape[1])))
model.add(LSTM(100, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

In [ ]:
model.fit(X_train, Y_train, epochs=10)

In [ ]:
import numpy as np

# Assume your DataFrame is called df

# Step 1: Convert DataFrame to a numpy array
data = X_train
print(np.shape(X_train))
# Step 2: Define the window size, i.e., the number of time steps to look back
window_size = 30

# Step 3: Create empty lists to hold the inputs and outputs
X_train = []
Y_train = np.array(Y_train[window_size:])

# Step 4: Loop through the data and create the inputs and outputs
for i in range(window_size, len(data)):
    X_train.append(data[i-window_size:i])
#    y.append(data[len(features)-1][0])  # use the close price as the output variable

# Step 5: Convert the lists to numpy arrays
X_train = np.array(X_train)

# Step 6: Reshape the input data into a 3D tensor
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], len(features)))
#X_train = np.transpose(X_train, (0, 2,1))


#X_test = data[-(TEST_SIZE+window_size): , : ]
X_test = []
Y_test = np.array(Y_test)

# Step 4: Loop through the data and create the inputs and outputs
for i in range(len(data)-TEST_SIZE, len(data)):
    X_test.append(data[i-window_size:i])

X_test = np.array(X_test)
#X_test = np.transpose(X_test, (0, 2,1))

print(np.shape(X_train), np.shape(Y_train))
print(np.shape(X_test), np.shape(Y_test))



In [ ]:
print(np.shape(X_train[0][0]))
np.shape(X_train[0])

In [ ]:


#model = Sequential()
#model.add(LSTM(50, return_sequences=True, kernel_regularizer=regularizers.l2(0.01),  kernel_initializer=initializers.RandomNormal(stddev=0.01), input_shape=(X_train.shape[1], X_train.shape[2])))
#model.add(LSTM(200, return_sequences=False))
#model.add(Dense(25))
#model.add(Dense(1, activation='sigmoid'))

opt = tf.keras.optimizers.experimental.SGD(learning_rate=.1)

model = Sequential()
model.add(LSTM(20, return_sequences=True, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=0.01), input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(50,return_sequences=True, activation='relu'))
model.add(LSTM(50,return_sequences=False, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='mse', optimizer=opt)

model.summary()

model.compile(optimizer=opt, loss=['binary_crossentropy'], metrics=['AUC','accuracy'])
history = model.fit(X_train, Y_train, batch_size=10, epochs=100, use_multiprocessing=True)


"""import tensorflow as tf

model = Sequential()
model.add(Bidirectional(
    LSTM(500, input_shape=(X_train.shape[1], X_train.shape[2]))
    )
)
model.add(LSTM(200, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=tf.keras.optimizers.experimental.Adam(
    learning_rate=0.02), loss='mean_squared_error', metrics=['accuracy'])


model.fit(X_train, Y_train, batch_size= 30, epochs=50, shuffle=False)"""

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy    
plt.plot(history.history['accuracy'])
plt.plot(history.history['auc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuracy', 'AUC'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
y_pr = model.predict(X_train)
y_pr

In [ ]:
predictions = model.predict(X_test)
predictions = predictions[:,0]
predictions

In [ ]:
predictions = (predictions-np.min(predictions))/(np.max(predictions)-np.min(predictions))
predictions

In [ ]:
#my_functions.plt_correlation(train)

In [ ]:
from sklearn.metrics import fbeta_score

best_precision = 0
thresh = 0
for i in np.arange(0.01, 1, 0.01):
    y_pred = [1 if el > i else 0 for el in predictions]
    #precision = sklearn.metrics.accuracy_score(Y_test, y_pred) # average='weighted', beta=.5)
    precision = sklearn.metrics.fbeta_score(Y_test, y_pred, average='weighted', beta=.5)
    if best_precision < precision:
        thresh = i 
        best_precision = precision

print(best_precision)
print(thresh)

In [ ]:
y_pred = [1 if el>thresh else 0 for el in predictions]

In [ ]:
from sklearn.metrics import confusion_matrix

def cm_analysis(
    y_true, 
    y_pred, 
    title: str,
    filename: str, 
    labels: List[str], 
    ymap=None, 
    figsize=(10,10)
) -> None:
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.

    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      title:     plot name
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.

    Returns:
        None
    """

    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.2f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.2f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    cm_perc = pd.DataFrame(cm_perc, index=labels, columns=labels)
    cm_perc.index.name = 'Actual'
    cm_perc.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(cm_perc, annot=annot, fmt='', linewidths=1, ax=ax)
    plt.title('\n'+title+'\n', fontsize=14)
    plt.show()


cm_analysis(Y_test, y_pred,title='asdasda',filename='cm.png',labels=[1,0])

In [ ]:
fpr_test, tpr_test, _ = sklearn.metrics.roc_curve(Y_test, predictions)
auc_test = sklearn.metrics.auc(fpr_test, tpr_test)
print(auc_test)
#skplt.metrics.plot_roc(df_test['signal'],y_probas, title=f"ROC curve, AUC=test: {auc_test:.4f}", classes_to_plot=[1], figsize=(10,10))
#plt.show()

In [ ]:
df.head()

In [ ]:
close_prices = df['price_close']
values = close_prices.values
training_data_len = int(len(values)* 0.8)

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(values.reshape(-1,1))
train_data = scaled_data[0: training_data_len, :]



x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

In [ ]:

x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

print(np.shape(x_train), np.shape(y_train))

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
print(np.shape(x_train), np.shape(y_train))

test_data = scaled_data[training_data_len-60: , : ]
x_test = []
y_test = values[training_data_len:]

for i in range(60, len(test_data)):
  x_test.append(test_data[i-60:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

print(np.shape(x_test), np.shape(y_test))

In [ ]:
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(100, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size= 1, epochs=3)

In [ ]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse

In [ ]:
data = df.filter(['price_close'])
train = data[:training_data_len]
validation = data[training_data_len:]
validation['Predictions'] = predictions
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date')
plt.ylabel('Close Price USD ($)')
plt.plot(train)
plt.plot(validation[['price_close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

In [ ]:
import numpy as np

# Assume your DataFrame is called df

# Step 1: Convert DataFrame to a numpy array
data = np.array(df[features])

# Step 2: Define the window size, i.e., the number of time steps to look back
window_size = 50

# Step 3: Create empty lists to hold the inputs and outputs
X = []
y = df['signal'][window_size:]

# Step 4: Loop through the data and create the inputs and outputs
for i in range(window_size, len(data)):
    X.append(data[i-window_size:i])
#    y.append(data[len(features)-1][0])  # use the close price as the output variable

# Step 5: Convert the lists to numpy arrays
X = np.array(X)
y = np.array(y)

# Step 6: Reshape the input data into a 3D tensor
X = np.reshape(X, (X.shape[0], X.shape[1], len(features)))
X2 = np.transpose(X, (0, 2,1))
print(np.shape(X), np.shape(y))
print(np.shape(X2))




In [ ]:
y

In [ ]:
from keras.layers.core import Dense, Flatten, Dropout
from keras.optimizers import Adagrad


model = Sequential()
model.add(LSTM(200, return_sequences=True, activation='relu', input_shape=(X2.shape[1],X2.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_squared_error',
              optimizer='rmsprop',    
              metrics=['accuracy']
            )

model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X2, y, batch_size= 1, epochs=3)

In [ ]:
y_test = model.predict(X2)

In [ ]:
y_test